In [1]:
import pandas as pd
import numpy as np
import re
import math
import requests
import json
import os
import statistics as st
import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.options.display.max_columns = None

In [3]:
# first dataset
df = pd.read_csv('../src/df_dpe_geokey_221129.csv')

In [4]:
df.sample(10)

,Unnamed: 0,annee_dpe,diag_dpe,type_bat_dpe,annee_construction_dpe,surface_habitable_dpe,num_rue_dpe,rue_dpe,code_postal_dpe,etage_dpe,nbe_niveaux_dpe,orientation_dpe,adresse_dpe,geokey
8750,8750,2016,D,Habitation (parties privatives d immeuble col,1974.0,34.96,27,AVENUE LACASSAGNE,69000,3.0,1.0,NaN,27 AVENUE LACASSAGNE,0.0
56822,56822,2022,E,appartement,2012.0,22.50,3,Rue Béchevelin,69007,1.0,1.0,NaN,3 Rue Béchevelin,69387_0765_00003
29230,29230,2017,C,NaN,1969.0,70.00,32,Rue Montesquieu,69007,NaN,0.0,NaN,32 Rue Montesquieu,NaN
39678,39678,2022,E,appartement,1960.0,52.30,65,Rue de lâ€ Abondance,69003,NaN,1.0,NaN,65 Rue de lâ€ Abondance,69383_0030_00065
57123,57123,2022,E,appartement,1947.0,35.50,29,Rue de Marseille,69007,1.0,1.0,NaN,29 Rue de Marseille,69387_4600_00029
12019,12019,2017,G,NaN,1948.0,66.46,NaN,nan 1 B RUE PROFESSEUR MARCEL DARGENT,69000,NaN,1.0,EO,nan nan 1 B RUE PROFESSEUR MARCEL DARGENT,0.0
45568,45568,2021,E,appartement,1948.0,117.10,6bis,Avenue Debrousse,69005,2.0,1.0,NaN,6bis Avenue Debrousse,69385_2115_00006_bis
16885,16885,2019,D,Habitation (parties privatives d immeuble col,1974.0,89.55,17 bis,Rue Thenard - Allee C,69000,1.0,1.0,NaN,17 bis Rue Thenard - Allee C,0.0
60901,60901,2022,D,appartement,1947.0,55.00,5,Place du 11 Novembre 1918,69008,3.0,1.0,NaN,5 Place du 11 Novembre 1918,69388_a8fj9m_00005
5057,5057,2015,D,NaN,1947.0,101.89,NaN,nan 55 Rue Garibaldi,69000,NaN,1.0,N,nan nan 55 Rue Garibaldi,0.0


In [5]:
# check the number of flats at the same adresses
df_group = df['geokey'].value_counts()

In [6]:
# check places where there are only 1 flat in the dataset
mask_index = df_group[lambda x: x==1].index.tolist()

In [7]:
len(mask_index)

4594

In [8]:
df_keep_dpe = df[df['geokey'].isin(mask_index)]

In [9]:
# number of rows to small to keep only the places with 1 flat (it would have been simpler to do the join with the 2 datasets)
len(df_keep_dpe)

4594

In [10]:
# second dataset
dg = pd.read_csv('../src/df_etalab_geokey_221130.csv')

In [11]:
dg.head()

,Unnamed: 0.1,Unnamed: 0,index,valeur_fonciere,adresse_numero,adresse_nom_voie,nom_commune,id_parcelle,type_local,surface_reelle_bati,nombre_pieces_principales,longitude,latitude,year,month,trim,index_prix,tuple_gps,closest_school_(m),closest_school_position,closest_bakery_(m),closest_bakery_position,closest_supermarket_(m),closest_supermarket_position,closest_metro_(m),closest_metro_position,closest_tram_(m),closest_tram_position,code_postal,adresse,geokey
0,0,10026,24429,6196748.0,71.0,RUE MONTAGNY,Lyon 8e Arrondissement,69388000CK0097,Appartement,4.0,1.0,4.850547,45.733555,2018,10,2018-T4,3.4,"(45.733555, 4.850547)",280.70,"(45.7346644003816, 4.8537957)",277.67,"(45.7353340003816, 4.8530576)",312.26,"(45.7357634003815, 4.8530323)",1120.01,"(45.737541, 4.837294)",490.44,"(45.72953, 4.853131)",69008,71 RUE MONTAGNY,69388_4850_00071
1,1,14588,94779,26138036.0,7.0,RUE DE LA REPUBLIQUE,Lyon 1er Arrondissement,69381000AS0064,Appartement,20.0,1.0,4.836266,45.766464,2019,9,2019-T3,4.0,"(45.766464, 4.836266)",63.83,"(45.767023968382, 4.83644685592725)",143.52,"(45.7672747003771, 4.8348262)",119.18,"(45.7653965003773, 4.8364032)",148.20,"(45.767785, 4.836012)",971.12,"(45.75888, 4.842474)",69001,7 RUE DE LA REPUBLIQUE,69381_6005_00007
2,2,26384,187992,25025000.0,18.0,RUE DE L ARBRE SEC,Lyon 1er Arrondissement,69381000AS0078,Appartement,27.0,1.0,4.835651,45.766366,2020,12,2020-T4,6.4,"(45.766366, 4.835651)",95.73,"(45.767023968382, 4.83644685592725)",119.59,"(45.7672747003771, 4.8348262)",110.04,"(45.7664184003772, 4.8342344)",160.25,"(45.767785, 4.836012)",986.43,"(45.75888, 4.842474)",69001,18 RUE DE L ARBRE SEC,69381_0425_00018
3,3,17591,102152,23332406.0,4.0,RUE ETIENNE RICHERAND,Lyon 3e Arrondissement,69383000EI0081,Appartement,26.0,1.0,4.866821,45.763442,2019,4,2019-T2,4.3,"(45.763442, 4.866821)",242.84,"(45.7656024197006, 4.86636338305401)",214.96,"(45.7638234003775, 4.8641044)",37.08,"(45.7635565003776, 4.8663721)",699.31,"(45.767025, 4.859412)",384.09,"(45.764314, 4.86203)",69003,4 RUE ETIENNE RICHERAND,69383_2595_00004
4,4,14388,94209,17473294.0,24.0,RUE NEUVE,Lyon 2e Arrondissement,69382000AC0045,Appartement,21.0,1.0,4.836262,45.765118,2019,9,2019-T3,4.0,"(45.765118, 4.836262)",97.82,"(45.7652058788412, 4.8375167949069)",241.73,"(45.7661227003772, 4.8334984)",14.10,"(45.7650821003774, 4.8360876)",194.91,"(45.763393, 4.835816)",844.60,"(45.75888, 4.842474)",69002,24 RUE NEUVE,69382_5005_00024


In [12]:
# check the number of flats at the same adresses
dg_group = dg['geokey'].value_counts()

In [13]:
# check places where there are only 1 flat per address in the dataset
mask_index = dg_group[lambda x: x==1].index.tolist()

In [14]:
len(mask_index)

2707

In [15]:
dg_keep_dpe = dg[dg['geokey'].isin(mask_index)]

In [16]:
# number of rows to small to keep only the places with 1 flat (it would have been simpler to do the join with the 2 datasets)
len(dg_keep_dpe)

2707

In [17]:
# give the list of all the geokeys present in the dataset
dg_temp = dg.groupby('geokey').first()

In [18]:
len(dg_temp) # 10 510 different geokeys detected

10510

In [19]:
mask_index3 = dg_group[lambda x: x==3].index.tolist() # explore the data with places where there are 3 flats at the same address

In [20]:
len(mask_index3)

1589

In [21]:
dg_mask_index3 = dg[dg['geokey'].isin(mask_index3)]

In [22]:
len(dg_mask_index3)

4767

In [23]:
dg_mask_index3.sort_values(by = 'geokey') # dataset of the places where there are 3 flats at the same address

,Unnamed: 0.1,Unnamed: 0,index,valeur_fonciere,adresse_numero,adresse_nom_voie,nom_commune,id_parcelle,type_local,surface_reelle_bati,nombre_pieces_principales,longitude,latitude,year,month,trim,index_prix,tuple_gps,closest_school_(m),closest_school_position,closest_bakery_(m),closest_bakery_position,closest_supermarket_(m),closest_supermarket_position,closest_metro_(m),closest_metro_position,closest_tram_(m),closest_tram_position,code_postal,adresse,geokey
30540,30540,189,576,425000.0,6.0,RUE D'ALGERIE,Lyon 1er Arrondissement,69381000AT0108,Appartement,96.0,4.0,4.831208,45.767127,2018,1,2018-T1,3.8,"(45.767127, 4.831208)",112.51,"(45.7677750863425, 4.83009417137382)",66.83,"(45.7667544003771, 4.8318841)",29.13,"(45.767359000377, 4.8310335)",379.75,"(45.767785, 4.836012)",1266.77,"(45.75888, 4.842474)",69001,6 RUE D'ALGERIE,69381_0160_00006
13276,13276,23520,178377,592550.0,6.0,RUE D'ALGERIE,Lyon 1er Arrondissement,69381000AT0108,Appartement,96.0,4.0,4.831208,45.767127,2020,3,2020-T1,6.4,"(45.767127, 4.831208)",112.51,"(45.7677750863425, 4.83009417137382)",66.83,"(45.7667544003771, 4.8318841)",29.13,"(45.767359000377, 4.8310335)",379.75,"(45.767785, 4.836012)",1266.77,"(45.75888, 4.842474)",69001,6 RUE D'ALGERIE,69381_0160_00006
11690,11690,33832,266965,440800.0,6.0,RUE D'ALGERIE,Lyon 1er Arrondissement,69381000AT0108,Appartement,68.0,3.0,4.831208,45.767127,2021,6,2021-T2,4.9,"(45.767127, 4.831208)",112.51,"(45.7677750863425, 4.83009417137382)",66.83,"(45.7667544003771, 4.8318841)",29.13,"(45.767359000377, 4.8310335)",379.75,"(45.767785, 4.836012)",1266.77,"(45.75888, 4.842474)",69001,6 RUE D'ALGERIE,69381_0160_00006
7761,7761,34946,272059,203460.0,8.0,RUE D'ALGERIE,Lyon 1er Arrondissement,69381000AT0107,Appartement,24.0,1.0,4.831262,45.767147,2021,10,2021-T4,4.6,"(45.767147, 4.831262)",108.55,"(45.7674802733547, 4.83257729341139)",65.07,"(45.7667544003771, 4.8318841)",29.49,"(45.767359000377, 4.8310335)",375.21,"(45.767785, 4.836012)",1265.50,"(45.75888, 4.842474)",69001,8 RUE D'ALGERIE,69381_0160_00008
14693,14693,42348,367149,166500.0,8.0,RUE D'ALGERIE,Lyon 1er Arrondissement,69381000AT0107,Appartement,28.0,2.0,4.831262,45.767147,2017,7,2017-T3,4.5,"(45.767147, 4.831262)",108.55,"(45.7674802733547, 4.83257729341139)",65.07,"(45.7667544003771, 4.8318841)",29.49,"(45.767359000377, 4.8310335)",375.21,"(45.767785, 4.836012)",1265.50,"(45.75888, 4.842474)",69001,8 RUE D'ALGERIE,69381_0160_00008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49433,49433,11781,85907,70000.0,13.0,GR GDE RUE DE VAISE,Lyon 9e Arrondissement,69389000BL0068,Appartement,40.0,1.0,4.806851,45.774850,2019,1,2019-T1,3.8,"(45.77485, 4.806851)",237.03,"(45.7737969882936, 4.80419372969254)",41.38,"(45.774769000376, 4.8063303)",129.25,"(45.7740912003761, 4.8081135)",103.61,"(45.774588, 4.805569)",3216.58,"(45.749554, 4.826965)",69009,13 GR GDE RUE DE VAISE,69389_7225_00013
42357,42357,12610,88225,130000.0,13.0,GR GDE RUE DE VAISE,Lyon 9e Arrondissement,69389000BL0068,Appartement,40.0,1.0,4.806851,45.774850,2019,4,2019-T2,4.3,"(45.77485, 4.806851)",237.03,"(45.7737969882936, 4.80419372969254)",41.38,"(45.774769000376, 4.8063303)",129.25,"(45.7740912003761, 4.8081135)",103.61,"(45.774588, 4.805569)",3216.58,"(45.749554, 4.826965)",69009,13 GR GDE RUE DE VAISE,69389_7225_00013
16883,16883,24346,180785,141850.0,36.0,GR GDE RUE DE VAISE,Lyon 9e Arrondissement,69389000BL0116,Appartement,25.0,1.0,4.807926,45.773879,2020,6,2020-T2,7.0,"(45.773879, 4.807926)",256.74,"(45.771591986586, 4.8083807425231)",80.16,"(45.7740825003761, 4.8069345)",27.72,"(45.7740912003761, 4.8081135)",199.08,"(45.774588, 4.805569)",3081.79,"(45.749554, 4.826965)",69009,36 GR GDE RUE DE VAISE,69389_7225_00036
15494,15494,25782,185814,140000.0,36.0,GR GDE RUE DE VAISE,Lyon 9e Arrondissement,69389000BL0116,Appartement,24.0,1.0,4.807926,45.773879,2020,11,2020-T4,6.4,"(45.773879, 4.807926)",256.74,"(45.771591986586, 4.8

In [24]:
geokey = '69389_7225_00036' # let's take the example with this particular geokey (3 last rows of the previous table for the sold flats at 36 GR GDE RUE DE VAISE)

In [25]:
mask_df = df['geokey'] == geokey

In [26]:
df_sample = df[mask_df]

In [27]:
df_sample # data of the example for the energetical diagnostics (5 rows)

,Unnamed: 0,annee_dpe,diag_dpe,type_bat_dpe,annee_construction_dpe,surface_habitable_dpe,num_rue_dpe,rue_dpe,code_postal_dpe,etage_dpe,nbe_niveaux_dpe,orientation_dpe,adresse_dpe,geokey
65563,65563,2021,E,appartement,1947.0,49.0,36,Grande Rue de Vaise,69009,3.0,1.0,NaN,36 Grande Rue de Vaise,69389_7225_00036
66944,66944,2022,D,appartement,1947.0,34.2,36,Grande Rue de Vaise,69009,1.0,1.0,NaN,36 Grande Rue de Vaise,69389_7225_00036
67497,67497,2022,C,appartement,1947.0,44.5,36,Grande Rue de Vaise,69009,3.0,1.0,NaN,36 Grande Rue de Vaise,69389_7225_00036
67591,67591,2022,D,appartement,1900.0,19.0,36,Grande Rue de Vaise,69009,NaN,1.0,NaN,36 Grande Rue de Vaise,69389_7225_00036
67628,67628,2022,G,appartement,1947.0,26.8,36,Grande Rue de Vaise,69009,1.0,1.0,NaN,36 Grande Rue de Vaise,69389_7225_00036


We can see that there are no correspondance between the 2 datasets. Indeed, we cannot match one flat with one diagnostic because even though they are at the same address, they don't have the same floor numbers, surfaces, years between the 2 datasets.

In [28]:
lf = df['geokey'].astype('string').unique()
lg = dg['geokey'].astype('string').unique()

In [29]:
lf[:10]

<StringArray>
[                 '0.0', '69381_1105_00013_bis',     '69381_2425_00014',
                    '0',     '69381_1155_00048',     '69381_3120_00008',
     '69381_3120_00002',     '69381_1585_00009',     '69381_7430_00003',
     '69381_5155_00017']
Length: 10, dtype: string

In [30]:
lg[:10]

<StringArray>
['69388_4850_00071', '69381_6005_00007', '69381_0425_00018',
 '69383_2595_00004', '69382_5005_00024',       '69387_0035',
 '69382_6005_00012', '69381_6005_00010', '69385_6255_00007',
 '69389_1665_00038']
Length: 10, dtype: string

In [31]:
# number of geokeys common to the 2 datasets

matches = set(lf) & set(lg)
len(matches)

6497

In [34]:
# we decide to take one unique hypothesis per building for each address : we take the mode to aggregate the rows at the same address. We assume that the result of the energetical diagnosis should be the same foe the whole building, as well as the year of the building.
df_agg = df.groupby(['geokey']).agg(lambda x: st.mode(x)).reset_index()

In [35]:
df_agg.sample(10)

,geokey,Unnamed: 0,annee_dpe,diag_dpe,type_bat_dpe,annee_construction_dpe,surface_habitable_dpe,num_rue_dpe,rue_dpe,code_postal_dpe,etage_dpe,nbe_niveaux_dpe,orientation_dpe,adresse_dpe
10553,69389_3680_00034,66314,2022,E,appartement,1948.0,63.3,34,Quai Jayr,69009,NaN,1.0,NaN,34 Quai Jayr
4387,69384_2400_00021,43651,2021,E,appartement,1947.0,50.8,21,Rue Dumont,69004,0.0,2.0,NaN,21 Rue Dumont
1452,69382_5720_00093,34353,2022,C,appartement,1947.0,90.3,93,Rue Président Edouard Herriot,69002,5.0,1.0,NaN,93 Rue Président Edouard Herriot
10500,69389_2545_00004,66100,2022,D,appartement,1970.0,104.0,4,Rue Ernest Fabrègue,69009,NaN,1.0,NaN,4 Rue Ernest Fabrègue
4658,69384_3925_00006,44624,2022,F,appartement,1947.0,39.0,6,Rue Joséphin Soulary,69004,3.0,1.0,NaN,6 Rue Joséphin Soulary
933,69382_1290_00015,25661,2021,D,NaN,1947.0,37.0,NaN,nan 15 Rue Casimir Périer,69002,NaN,1.0,NaN,nan nan 15 Rue Casimir Périer
375,69381_4140_00029,32444,2022,D,appartement,1947.0,36.0,29,Rue Lanterne,69001,NaN,1.0,NaN,29 Rue Lanterne
113,69381_1105_00022,32666,2022,D,appartement,1989.0,55.0,22,Rue Bouteille,69001,1.0,1.0,NaN,22 Rue Bouteille
720,69382_0230_00006,33919,2022,E,appartement,1947.0,67.3,6,Place Ampère,69002,1.0,1.0,NaN,6 Place Ampère
10229,69389_0073_00001,65326,2021,D,appartement,2006.0,80.0,1,Place Abbé Pierre,69009,NaN,1.0,NaN,1 Place Abbé Pierre


In [36]:
# we apply the unique diagnosis and year of building for each real estate sold in the Etalab dataset. By doing this, we loose the orientation and the floor number features.
# left join
df_model_left = dg.merge(df_agg, on='geokey', how='left')

In [37]:
df_model_left.sample(10)

,Unnamed: 0.1,Unnamed: 0_x,index,valeur_fonciere,adresse_numero,adresse_nom_voie,nom_commune,id_parcelle,type_local,surface_reelle_bati,nombre_pieces_principales,longitude,latitude,year,month,trim,index_prix,tuple_gps,closest_school_(m),closest_school_position,closest_bakery_(m),closest_bakery_position,closest_supermarket_(m),closest_supermarket_position,closest_metro_(m),closest_metro_position,closest_tram_(m),closest_tram_position,code_postal,adresse,geokey,Unnamed: 0_y,annee_dpe,diag_dpe,type_bat_dpe,annee_construction_dpe,surface_habitable_dpe,num_rue_dpe,rue_dpe,code_postal_dpe,etage_dpe,nbe_niveaux_dpe,orientation_dpe,adresse_dpe
11696,11696,32242,260913,700000.0,23.0,RUE DU CLOS SAVARON,Lyon 4e Arrondissement,69384000AR0019,Appartement,108.0,4.0,4.824675,45.782388,2021,2,2021-T1,5.1,"(45.782388, 4.824675)",167.38,"(45.7808896935656, 4.82446805509733)",351.24,"(45.7796741003753, 4.8269926)",452.43,"(45.7792229716282, 4.82834128383433)",380.57,"(45.779586, 4.827493)",2943.92,"(45.760123, 4.845212)",69004,23 RUE DU CLOS SAVARON,69384_1760_00023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35903,35903,8290,21165,339200.0,20.0,RUE CHATEAUBRIAND,Lyon 7e Arrondissement,69387000BS0090,Appartement,86.0,4.0,4.838962,45.735990,2018,8,2018-T3,3.4,"(45.73599, 4.838962)",165.04,"(45.7363670003814, 4.8369052)",73.57,"(45.7361770555883, 4.83987124616591)",272.27,"(45.7374909003812, 4.8361901)",215.64,"(45.737541, 4.837294)",693.91,"(45.731075, 4.833453)",69007,20 RUE CHATEAUBRIAND,69387_1565_00020,53240.0,2021.0,C,appartement,2001.0,73.6,20,Rue Chateaubriand,69007.0,4.0,1.0,NaN,20 Rue Chateaubriand
24205,24205,47711,379995,388000.0,40.0,RUE PRE GAUDRY,Lyon 7e Arrondissement,69387000BS0185,Appartement,78.0,4.0,4.835817,45.738516,2017,12,2017-T4,4.3,"(45.738516, 4.835817)",245.71,"(45.7389840808393, 4.83891126033237)",166.61,"(45.7372059003813, 4.8368587)",117.61,"(45.7374909003812, 4.8361901)",157.78,"(45.737541, 4.837294)",847.50,"(45.731075, 4.833453)",69007,40 RUE PRE GAUDRY,69387_5695,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23265,23265,29911,196667,519900.0,9.0,RUE GUSTAVE NADAUD,Lyon 7e Arrondissement,69387000BO0100,Appartement,103.0,4.0,4.832331,45.744439,2020,9,2020-T3,6.6,"(45.744439, 4.832331)",326.59,"(45.7416335176773, 4.83357663119815)",335.10,"(45.7457553003801, 4.8362157)",372.44,"(45.7474972003798, 4.8342886)",704.88,"(45.749554, 4.826965)",423.83,"(45.747056, 4.836302)",69007,9 RUE GUSTAVE NADAUD,69387_3425_00009,52875.0,2021.0,D,appartement,1989.0,70.9,9,Rue Gustave Nadaud,69007.0,3.0,1.0,NaN,9 Rue Gustave Nadaud
17088,17088,39615,302769,560000.0,10.0,RUE ST-PHILIPPE,Lyon 3e Arrondissement,69383000BD0080,Appartement,99.0,5.0,4.865582,45.750197,2021,8,2021-T3,5.3,"(45.750197, 4.865582)",155.15,"(45.7506839500737, 4.86745588896569)",44.48,"(45.7502514003795, 4.865014)",264.68,"(45.7478541003798, 4.8649798)",287.39,"(45.747627, 4.865189)",391.60,"(45.752938, 4.868751)",69003,10 RUE ST-PHILIPPE,69383_6520_00010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3222,3222,27964,191935,3250000.0,33.0,RUE DES RANCY,Lyon 3e Arrondissement,69383000AT0095,Appartement,64.0,2.0,4.852149,45.756233,2020,4,2020-T2,7.0,"(45.756233, 4.852149)",140.81,"(45.7556757941366, 4.8537788475716)",130.39,"(45.7569563003785, 4.8508261)",154.35,"(45.7565794003786, 4.8540756)",483.30,"(45.753799, 4.846988)",513.84,"(45.760708, 4.853801)",69003,33 RUE DES RANCY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8941,8941,49532,423918,231000.0,19.0,RUE DU BOEUF,Lyon 5e Arrondissement,69385000AH0075,Appartement,31.0,1.0,4.826932,45.762404,2022,5,2022-T2,4.5,"(45.762404, 4.826932)",189.70,"(45.760772500378, 4.8262171)",59.75,"(45.7624720003777, 4.8276961)",381.23,"(45.7614924003779, 4.8316695)",274.69,"(45.759963, 4.826388)",1267.77,"(45.75888, 4.842474)",69005,19 RUE DU BOEUF,69385_0950_00019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21611,21611,38766,296641,130000.0,128.0,RUE BARABAN,Lyon 3e Arrondissement,69383000DS

In [38]:
# inner join
df_model_inner = dg.merge(df_agg, on='geokey', how='inner')

In [39]:
df_model_inner.sample(10)

,Unnamed: 0.1,Unnamed: 0_x,index,valeur_fonciere,adresse_numero,adresse_nom_voie,nom_commune,id_parcelle,type_local,surface_reelle_bati,nombre_pieces_principales,longitude,latitude,year,month,trim,index_prix,tuple_gps,closest_school_(m),closest_school_position,closest_bakery_(m),closest_bakery_position,closest_supermarket_(m),closest_supermarket_position,closest_metro_(m),closest_metro_position,closest_tram_(m),closest_tram_position,code_postal,adresse,geokey,Unnamed: 0_y,annee_dpe,diag_dpe,type_bat_dpe,annee_construction_dpe,surface_habitable_dpe,num_rue_dpe,rue_dpe,code_postal_dpe,etage_dpe,nbe_niveaux_dpe,orientation_dpe,adresse_dpe
23730,49274,15575,97994,75000.0,8.0,RUE AUGUSTE COMTE,Lyon 2e Arrondissement,69382000AR0007,Appartement,41.0,1.0,4.831488,45.756026,2019,12,2019-T4,5.3,"(45.756026, 4.831488)",133.02,"(45.7549137067992, 4.83211904353441)",73.85,"(45.7555265003787, 4.8308606)",200.76,"(45.7568602003785, 4.8337829)",202.23,"(45.757542, 4.832928)",766.52,"(45.749655, 4.835261)",69002,8 RUE AUGUSTE COMTE,69382_0535_00008,33901,2021,C,appartement,1947.0,39.0,8,Rue Auguste Comte,69002,5.0,1.0,NaN,8 Rue Auguste Comte
8624,8803,37507,287437,489000.0,7.0,RUE HENRI PENSIER,Lyon 8e Arrondissement,69388000BR0045,Appartement,65.0,3.0,4.863419,45.745390,2021,5,2021-T2,4.9,"(45.74539, 4.863419)",218.70,"(45.7472264663328, 4.86442806802367)",73.94,"(45.7458395003801, 4.8641211)",70.51,"(45.7458588003801, 4.8640309)",284.14,"(45.747627, 4.865189)",339.80,"(45.745802, 4.85908)",69008,7 RUE HENRI PENSIER,69388_3520_00007,60184,2022,E,appartement,1920.0,28.5,7,Rue Henri Pensier,69008,NaN,1.0,NaN,7 Rue Henri Pensier
13163,28471,4299,13019,621000.0,16.0,RUE FELIX MANGINI,Lyon 9e Arrondissement,69389000AL0138,Appartement,135.0,5.0,4.817780,45.790439,2018,12,2018-T4,3.4,"(45.790439, 4.81778)",259.96,"(45.7902949779062, 4.81443356072397)",184.59,"(45.7909365003737, 4.8155087)",981.23,"(45.7840965003746, 4.8089816)",1296.89,"(45.785407, 4.832869)",3909.55,"(45.780873, 4.866297)",69009,16 RUE FELIX MANGINI,69389_2690_00016,64981,2021,C,appartement,2016.0,65.3,16,Rue Félix Mangini,69009,2.0,1.0,NaN,16 Rue Félix Mangini
5820,11644,2181,6469,130000.0,6.0,RUE TONY TOLLET,Lyon 2e Arrondissement,69382000AP0002,Appartement,20.0,1.0,4.828822,45.756389,2018,7,2018-T3,3.4,"(45.756389, 4.828822)",29.02,"(45.7562571306657, 4.82914477545998)",83.45,"(45.7570784003785, 4.8283968)",72.61,"(45.7567167003785, 4.8280125)",343.38,"(45.757542, 4.832928)",773.55,"(45.749554, 4.826965)",69002,6 RUE TONY TOLLET,69382_7040_00006,35553,2022,D,appartement,2006.0,32.3,6,Rue Tony Tollet,69002,1.0,1.0,NaN,6 Rue Tony Tollet
30058,42634,9561,23648,251000.0,115.0,BD YVES FARGE,Lyon 7e Arrondissement,69387000BY0010,Appartement,78.0,4.0,4.828282,45.735707,2018,10,2018-T4,3.4,"(45.735707, 4.828282)",175.07,"(45.7349609609211, 4.83026840292077)",362.20,"(45.7373188003813, 4.8323375)",99.47,"(45.7365116130078, 4.82884205095171)",652.96,"(45.731075, 4.833453)",479.40,"(45.731417, 4.828896)",69007,115 BD YVES FARGE,69387_7575_00115,53832,2022,C,appartement,1948.0,78.2,115,Boulevard Yves Farge,69007,7.0,1.0,NaN,115 Boulevard Yves Farge
728,43642,9690,23875,202715.0,70.0,BD JEAN VINGT TROIS,Lyon 8e Arrondissement,69388000AS0041,Appartement,66.0,3.0,4.869751,45.737013,2018,11,2018-T4,3.4,"(45.737013, 4.869751)",131.02,"(45.7368133797379, 4.87141478522104)",134.76,"(45.7358060003815, 4.869595)",157.82,"(45.7367311003813, 4.867758)",944.58,"(45.745395, 4.871728)",120.66,"(45.736845, 4.868215)",69008,70 BD JEAN VINGT TROIS,0,25100,2022,C,appartement,1947.0,59.4,NaN,NaN,69007,1.0,1.0,NaN,nan nan
10135,47363,34151,268509,200000.0,10.0,AV DE MENIVAL,Lyon 5e Arrondissement,69385000BC0024,Appartement,83.0,5.0,4.786061,45.757444,2021,7,2021-T3,5.3,"(45.757444, 4.786061)",98.20,"(45.7566190003785, 4.7856092)",206.17,"(45.7564941003786, 4.7837788)",234.20,"(45.7564920003786, 4.7833682)",1798.00,"(45.766382, 4.805376)",3160.77,"(45.744213, 4.822116)",69005,10 AV DE MENIVAL,69385_4720_00010,4

In [40]:
# thanks to our hypothesis, we can keep a large amount of rows
len(df_model_left)

51125

In [41]:
len(df_model_inner)

32803

In [71]:
df_model_left.to_csv('../src/df_model_left_221201.csv')

In [72]:
df_model_inner.to_csv('../src/df_model_inner_221201.csv')